In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

import xgboost as xgb
import pickle

In [2]:
profile = pd.read_csv("data/profile.csv")
portfolio = pd.read_csv("data/portfolio-dummy.csv")
train_set = pd.read_csv("data/train.csv")
test_set = pd.read_csv("data/test.csv")

In [5]:
train_set.merge(profile, left_on='person', right_on='id').merge(portfolio, left_on='offer_id', right_on="id")

,person,offer_id,offer viewed,id_x,age,income,income_na,gender_F,gender_M,gender_O,...,reward,difficulty,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount,offer_type_informational
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,1.0,0009655768c64bdeb2e877511632db8f,33.000000,72000.000000,0,0,1,0,...,0,0,4,1.0,1.0,0.0,1.0,0,0,1
1,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,1.0,0011e0d4e6b944f998e987f904e8c1e5,40.000000,57000.000000,0,0,0,1,...,0,0,4,1.0,1.0,0.0,1.0,0,0,1
2,003d66b6608740288d6cc97a6903f4f0,3f207df678b143eea3cee63160fa8bed,1.0,003d66b6608740288d6cc97a6903f4f0,26.000000,73000.000000,0,1,0,0,...,0,0,4,1.0,1.0,0.0,1.0,0,0,1
3,004b041fbfe44859945daa2c7f79ee64,3f207df678b143eea3cee63160fa8bed,0.0,004b041fbfe44859945daa2c7f79ee64,55.000000,74000.000000,0,1,0,0,...,0,0,4,1.0,1.0,0.0,1.0,0,0,1
4,0069a50874d846438e58acff5e594725,3f207df678b143eea3cee63160fa8bed,0.0,0069a50874d846438e58acff5e594725,54.393524,65404.991568,1,0,0,0,...,0,0,4,1.0,1.0,0.0,1.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45370,ffd53e5ba61d41378b66efc00d9f9682,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,ffd53e5ba61d41378b66efc00d9f9682,40.000000,32000.000000,0,0,1,0,...,5,5,7,1.0,1.0,0.0,1.0,1,0,0
45371,ffdefcac307f4ca99ac1ebd51470f106,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,ffdefcac307f4ca99ac1ebd51470f106,87.000000,47000.000000,0,1,0,0,...,5,5,7,1.0,1.0,0.0,1.0,1,0,0
45372,ffe5257abf8840b395e1ee6b29894637,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0,ffe5257abf8840b395e1ee6b29894637,77.000000,75000.000000,0,0,1,0,...,5,5,7,1.0,1.0,0.0,1.0,1,0,0
45373,ffecb1f8543f4bf7bade023de366d6bf,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0,ffecb1f8543f4bf7bade023de366d6bf,54.393524,65404.991568,1,0,0,0,...,5,5,7,1.0,1.0,0.0,1.0,1,0,0


In [56]:
def encode_dataset(df, shuffle=False):
    df = df.merge(profile, left_on='person', right_on='id') \
        .merge(portfolio, left_on='offer_id', right_on="id")
    if shuffle:
        df = df.sample(frac=1)
    df.index = df[['person', 'offer_id']]
    
    y = df['offer viewed']
    df = df.drop(columns=['person', 'offer_id', 'id_x', 'id_y', 'index', 'offer viewed'])
    return df, y

X_train, y_train = encode_dataset(train_set, True)

In [57]:
X_train

,age,income,income_na,gender_F,gender_M,gender_O,age_na,member_since,reward,difficulty,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount,offer_type_informational
"(c527628db5064bcea197c2a1e28f7d7d, 2298d6c36e964ae4a3e7e9706d1fb8c2)",42.000000,49000.000000,0,0,1,0,0,2016,3,7,7,1.0,1.0,1.0,1.0,0,1,0
"(ed1418961ec34a12bed2a53bb712b58d, 2298d6c36e964ae4a3e7e9706d1fb8c2)",54.393524,65404.991568,1,0,0,0,1,2018,3,7,7,1.0,1.0,1.0,1.0,0,1,0
"(3ed995ed43a641c1a282760dab603f1f, 2906b810c7d4411798c6938adc9daaa5)",56.000000,57000.000000,0,0,1,0,0,2017,2,10,7,1.0,1.0,0.0,1.0,0,1,0
"(05b0cd60c07e4ffdb1829da2010b55ee, fafdcd668e3743c1bb461111dcafc2a4)",43.000000,52000.000000,0,0,1,0,0,2015,2,10,10,1.0,1.0,1.0,1.0,0,1,0
"(6d3b0836d650474aafc8aa0fb5f20cb3, 2906b810c7d4411798c6938adc9daaa5)",52.000000,62000.000000,0,1,0,0,0,2017,2,10,7,1.0,1.0,0.0,1.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(6b247ba48509456eb57b24f878f313ea, 9b98b8c7a33c4b65b9aebfe6a799e6d9)",28.000000,68000.000000,0,1,0,0,0,2017,5,5,7,1.0,1.0,0.0,1.0,1,0,0
"(9a35aada6b3447899a6a8eb4bc31aed9, 4d5c57ea9a6940dd891ad53e9dbe8da0)",64.000000,56000.000000,0,0,1,0,0,2018,10,10,5,1.0,1.0,1.0,1.0,1,0,0
"(bd06525874e148689e502cda5623b25e, 0b1e1539f2cc45b7b9fa7c272da2e1d7)",35.000000,66000.000000,0,0,1,0,0,2016,5,20,10,1.0,0.0,0.0,1.0,0,1,0
"(f129b6b8c71e482b8b273b14d06ffe26, 2298d6c36e964ae4a3e7e9706d1fb8c2)",46.000000,87000.000000,0,1,0,0,0,2017,3,7,7,1.0,1.0,1.0,1.0,0,1,0


In [58]:
y_train

(c527628db5064bcea197c2a1e28f7d7d, 2298d6c36e964ae4a3e7e9706d1fb8c2)    1.0
(ed1418961ec34a12bed2a53bb712b58d, 2298d6c36e964ae4a3e7e9706d1fb8c2)    1.0
(3ed995ed43a641c1a282760dab603f1f, 2906b810c7d4411798c6938adc9daaa5)    1.0
(05b0cd60c07e4ffdb1829da2010b55ee, fafdcd668e3743c1bb461111dcafc2a4)    1.0
(6d3b0836d650474aafc8aa0fb5f20cb3, 2906b810c7d4411798c6938adc9daaa5)    1.0
                                                                       ... 
(6b247ba48509456eb57b24f878f313ea, 9b98b8c7a33c4b65b9aebfe6a799e6d9)    1.0
(9a35aada6b3447899a6a8eb4bc31aed9, 4d5c57ea9a6940dd891ad53e9dbe8da0)    1.0
(bd06525874e148689e502cda5623b25e, 0b1e1539f2cc45b7b9fa7c272da2e1d7)    0.0
(f129b6b8c71e482b8b273b14d06ffe26, 2298d6c36e964ae4a3e7e9706d1fb8c2)    1.0
(4f3aab9a035e4e73b8b6842031ccddb4, 2298d6c36e964ae4a3e7e9706d1fb8c2)    1.0
Name: offer viewed, Length: 45375, dtype: float64

In [59]:
X_train, y_train = encode_dataset(train_set)
X_test, y_test = encode_dataset(test_set)

In [89]:
def get_baseline_model():
    pipe = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('linear_svc', LinearSVC(
                max_iter=100000
            ))
        ],
        verbose = True
    )
    
    return pipe

def get_randomforest():
    pipe = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('random_forest', RandomForestClassifier())
        ],
        verbose = True
    )
    
    return pipe

def get_svc():
    pipe = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('svc', SVC())
        ],
        verbose = True
    )
    
    return pipe

def get_xgb():
    pipe = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('xgb', xgb.XGBClassifier(
                silent=False,
                objective='binary:logistic',
                booster='gbtree'
            ))
        ],
        verbose = True
    )
    
    return pipe

def get_xgb_gcv():
    
    param_grid = {
        'xgb__booster' :['gbtree'],
        'xgb__silent':[False],
        'xgb__max_depth':[5,6,8],
        'xgb__min_child_weight':[1,3,5],
        'xgb__gamma':[0,1,2,3],
        'xgb__nthread':[4],
        'xgb__colsample_bytree':[0.5,0.8],
        'xgb__colsample_bylevel':[0.9],
        'xgb__n_estimators':[50],
        'xgb__objective':['binary:logistic'],
        'xgb__random_state':[2]
    }
    pipe = Pipeline(
        [
            ('scaler', StandardScaler()),
            ('xgb', xgb.XGBClassifier())
        ],
        verbose = True
    )
    gcv=GridSearchCV(pipe, param_grid=param_grid, scoring='f1', n_jobs=1, verbose=2)
    
    return gcv


In [69]:
def train_and_test(name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    print("Report for", name)
    print("Train Report>")
    print("Train Accuracy:", model.score(X_train, y_train))
    y_pred = model.predict(X_train)
    print(classification_report(y_train, y_pred))
    
    print("Test Report>")
    print("Test Accuracy:", model.score(X_test, y_test))
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

In [62]:
model = get_baseline_model()

In [47]:
%%time
train_and_test("baseline", model, X_train, y_train, X_test, y_test)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing linear_svc, total=  35.8s
Train Report>
Train Accuracy: 0.7970468319559229
              precision    recall  f1-score   support

         0.0       0.64      0.28      0.39     10469
         1.0       0.82      0.95      0.88     34906

    accuracy                           0.80     45375
   macro avg       0.73      0.62      0.63     45375
weighted avg       0.77      0.80      0.77     45375

Test Report>
Test Accuracy: 0.7824313853431752
              precision    recall  f1-score   support

         0.0       0.66      0.26      0.37      6102
         1.0       0.80      0.95      0.87     18419

    accuracy                           0.78     24521
   macro avg       0.73      0.61      0.62     24521
weighted avg       0.76      0.78      0.75     24521

Wall time: 36.1 s


In [63]:
models = [
    ("baseline", get_baseline_model()),
    ("random-forest", get_randomforest()),
    ("svc", get_svc())
]

In [64]:
%%time
for name, model in models:
    train_and_test(name, model, X_train, y_train, X_test, y_test)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing linear_svc, total=  38.7s
Report for baseline
Train Report>
Train Accuracy: 0.7970468319559229
              precision    recall  f1-score   support

         0.0       0.64      0.28      0.39     10469
         1.0       0.82      0.95      0.88     34906

    accuracy                           0.80     45375
   macro avg       0.73      0.62      0.63     45375
weighted avg       0.77      0.80      0.77     45375

Test Report>
Test Accuracy: 0.7824313853431752
              precision    recall  f1-score   support

         0.0       0.66      0.26      0.37      6102
         1.0       0.80      0.95      0.87     18419

    accuracy                           0.78     24521
   macro avg       0.73      0.61      0.62     24521
weighted avg       0.76      0.78      0.75     24521

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ..... 

In [70]:
xgbmodel = get_xgb()
train_and_test("xgb", xgbmodel, X_train, y_train, X_test, y_test)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.1s
[18:19:36] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   3.5s
Report for xgb
Train Report>
Train Accuracy: 0.8528484848484849
              precision    recall  f1-score   support

         0.0       0.73      0.58      0.65     10469
         1.0       0.88      0.93      0.91     34906

    accuracy                           0.85     45375
   macro avg       0.80      0.76      0.78     45375
weighted avg       0.85      0.85      0.85     45375

Test Report>
Test Accuracy: 0.8188491497084132
              precision    recall  f

In [90]:
xgb_gcv_model = get_xgb_gcv()
train_and_test("xgb_gcb", xgb_gcv_model, X_train, y_train, X_test, y_test)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.5, xgb__gamma=0, xgb__max_depth=5, xgb__min_child_weight=1, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False 
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[18:29:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.4s
[CV]  xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.5, xgb__gamma=0, xgb__max_depth=5, xgb__min_child_weight=1, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False, total=   0.5s
[CV] xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.5, xgb__gamma=0, xgb__max_depth=5, xgb__min_child_weight=1, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False 
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[18:29:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.4s
[CV]  xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.5, xgb__gamma=0, xgb__max_depth=5, xgb__min_child_weight=1, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False, total=   0.5s
[CV] xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.5, xgb__gamma=0, xgb__max_depth=5, xgb__min_child_weight=1, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False 
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[18:29:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open

...

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   1.0s
[CV]  xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.8, xgb__gamma=3, xgb__max_depth=6, xgb__min_child_weight=3, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False, total=   1.1s
[CV] xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.8, xgb__gamma=3, xgb__max_depth=6, xgb__min_child_weight=3, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False 
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[18:33:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   1.1s
[CV]  xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.8, xgb__gamma=3, xgb__max_depth=8, xgb__min_child_weight=1, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False, total=   1.2s
[CV] xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.8, xgb__gamma=3, xgb__max_depth=8, xgb__min_child_weight=1, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False 
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[18:33:30] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open

[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.8s
[CV]  xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.8, xgb__gamma=3, xgb__max_depth=8, xgb__min_child_weight=3, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False, total=   0.9s
[CV] xgb__booster=gbtree, xgb__colsample_bylevel=0.9, xgb__colsample_bytree=0.8, xgb__gamma=3, xgb__max_depth=8, xgb__min_child_weight=3, xgb__n_estimators=50, xgb__nthread=4, xgb__objective=binary:logistic, xgb__random_state=2, xgb__silent=False 
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[18:33:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:  4.1min finished


[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.6s
Report for xgb_gcb
Train Report>
Train Accuracy: 0.8946921029528574
              precision    recall  f1-score   support

         0.0       0.67      0.55      0.60     10469
         1.0       0.87      0.92      0.89     34906

    accuracy                           0.83     45375
   macro avg       0.77      0.73      0.75     45375
weighted avg       0.82      0.83      0.83     45375

Test Report>
Test Accuracy: 0.8848810237431258
              precision    recall  f1-score   support

         0.0       0.68      0.51      0.59      6102
         1.0       0.85      0.92      0.88     18419

    accuracy                           0.82     24521
   macro avg       0.77      0.72      0.74     24521
weighted avg       0.81      0.82      0.81     24521



In [92]:
with open("model/xgb-optimal.pickle","wb") as fw:
    pickle.dump(xgb_gcv_model, fw)